In [1]:
# install dependencies
!pip install -q --upgrade git+https://github.com/terrierteam/pyterrier_adaptive.git
!pip install -q --upgrade git+https://github.com/terrierteam/pyterrier_t5.git
!pip install -q pyterrier_pisa

     |████████████████████████████████| 97 kB 1.2 MB/s 
     |████████████████████████████████| 1.1 MB 63.0 MB/s 
     |████████████████████████████████| 69 kB 8.9 MB/s 
     |████████████████████████████████| 303 kB 70.1 MB/s 
     |████████████████████████████████| 46 kB 4.8 MB/s 
     |████████████████████████████████| 45 kB 3.2 MB/s 
     |████████████████████████████████| 1.2 MB 57.4 MB/s 
     |████████████████████████████████| 126 kB 71.8 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 294 kB 67.1 MB/s 
     |████████████████████████████████| 596 kB 67.2 MB/s 
     |████████████████████████████████| 72 kB 1.7 MB/s 
     |████████████████████████████████| 287 kB 50.0 MB/s 
     |████████████████████████████████| 59 kB 8.2 MB/s 
     |████████████████████████████████| 4.7 MB 9.7 MB/s 
     |████████████████████████████████| 1.3 MB 55.6 MB/s 
     |███████████

In [2]:
# imports
import pyterrier as pt
pt.init()
from pyterrier.measures import *
from pyterrier_adaptive import GAR, CorpusGraph
from pyterrier_t5 import MonoT5ReRanker
from pyterrier_pisa import PisaIndex

terrier-assemblies 5.6 jar-with-dependencies not found, downloading to /root/.pyterrier...
Done
terrier-python-helper 0.0.6 jar not found, downloading to /root/.pyterrier...
Done


PyTerrier 0.8.1 has loaded Terrier 5.6 (built by craigmacdonald on 2021-09-17 13:27)



In [3]:
# Create required components
dataset = pt.get_dataset('irds:msmarco-passage')
retriever = PisaIndex.from_dataset('msmarco_passage').bm25()
scorer = pt.text.get_text(dataset, 'text') >> MonoT5ReRanker(verbose=False, batch_size=16)
graph = CorpusGraph.from_dataset('msmarco_passage', 'corpusgraph_bm25_k16').to_limit_k(8)

/usr/local/lib/python3.7/dist-packages/pyterrier/datasets.py:278: UserWarning: Downloading index of > 2GB.
  warn("Downloading index of > 2GB.")


fwd.termlex:   0%|          | 0.00/20.6M [00:00<?, ?iB/s]

fwd.urls:   0%|          | 0.00/8.43M [00:00<?, ?iB/s]

pt_pisa_config.json:   0%|          | 0.00/22.0 [00:00<?, ?iB/s]

inv.freqs:   0%|          | 0.00/1.29G [00:00<?, ?iB/s]

...

docnos.npids:   0%|          | 0.00/206 [00:00<?, ?iB/s]

edges.u32.np:   0%|          | 0.00/540M [00:00<?, ?iB/s]

md5sums:   0%|          | 0.00/190 [00:00<?, ?iB/s]

pt_meta.json:   0%|          | 0.00/76.0 [00:00<?, ?iB/s]

weights.f16.np:   0%|          | 0.00/270M [00:00<?, ?iB/s]

In [4]:
# A simple example
pipeline = retriever >> GAR(scorer, graph) >> pt.text.get_text(dataset, 'text')

pipeline.search('clustering hypothesis information retrieval')

[INFO] [starting] building docstore
[INFO] Please confirm you agree to the MSMARCO data usage agreement found at <http://www.msmarco.org/dataset.aspx>
[INFO] If you have a local copy of https://msmarco.blob.core.windows.net/msmarcoranking/collectionandqueries.tar.gz, you can symlink it here to avoid downloading it again: /root/.ir_datasets/downloads/31644046b18952c1386cd4564ba2ae69
[INFO] [starting] https://msmarco.blob.core.windows.net/msmarcoranking/collectionandqueries.tar.gz
docs_iter:   0%|                                   | 0/8841823 [00:00<?, ?doc/s]
https://msmarco.blob.core.windows.net/msmarcoranking/collectionandqueries.tar.gz: 0.0%| 0.00/1.06G [00:00<?, ?B/s]
https://msmarco.blob.core.windows.net/msmarcoranking/collectionandqueries.tar.gz: 0.0%| 311k/1.06G [00:00<06:06, 2.88MB/s]
https://msmarco.blob.core.windows.net/msmarcoranking/collectionandqueries.tar.gz: 0.1%| 1.49M/1.06G [00:00<02:30, 7.04MB/s]
https://msmarco.blob.core.windows.net/msmarcoranking/collectionandqueries

,qid,query,docno,rank,score,iteration,text
0,1,clustering hypothesis information retrieval,2180710,0,-0.017042,0,Cluster analysis or clustering is the task of ...
1,1,clustering hypothesis information retrieval,8430269,1,-0.166527,1,Clustering is the grouping of a particular set...
2,1,clustering hypothesis information retrieval,1091429,2,-0.208341,1,Clustering is a fundamental data analysis meth...
3,1,clustering hypothesis information retrieval,2180711,3,-0.341067,5,Cluster analysis or clustering is the task of ...
4,1,clustering hypothesis information retrieval,6031959,4,-0.366815,5,Cluster analysis or clustering is the task of ...
...,...,...,...,...,...,...,...
995,1,clustering hypothesis information retrieval,2471954,995,-14.283018,33,Now we will calculate the distance to the loca...
996,1,clustering hypothesis information retrieval,5237256,996,-14.315647,22,Retrieve null date from table. 1 Re: Retrieve...
997,1,clustering hypothesis information retrieval,8058446,997,-14.336015,31,Modified Newtonian dynamics - Observational ev...
998,1,clustering hypothesis information retrieval,8828444,998,-14.383328,41,"Calculate mean/avaerge, range, median and mode..."


In [5]:
dataset = pt.get_dataset('irds:msmarco-passage/trec-dl-2019/judged')
pt.Experiment(
    [retriever, retriever >> scorer, retriever >> GAR(scorer, graph)],
    dataset.get_topics(),
    dataset.get_qrels(),
    [nDCG, MAP(rel=2), R(rel=2)@1000],
    names=['bm25', 'bm25 >> monot5', 'bm25 >> GAR(monot5)']
)

[INFO] [starting] https://trec.nist.gov/data/deep/2019qrels-pass.txt
[INFO] [finished] https://trec.nist.gov/data/deep/2019qrels-pass.txt: [00:00] [187kB] [1.08MB/s]
[INFO] [starting] https://msmarco.blob.core.windows.net/msmarcoranking/msmarco-test2019-queries.tsv.gz
[INFO] [finished] https://msmarco.blob.core.windows.net/msmarcoranking/msmarco-test2019-queries.tsv.gz: [00:00] [4.28kB] [1.17MB/s]


,name,nDCG,AP(rel=2),R(rel=2)@1000
0,bm25,0.602325,0.303099,0.755495
1,bm25 >> monot5,0.696573,0.482285,0.755495
2,bm25 >> GAR(monot5),0.724529,0.490023,0.825952
